file paths

In [8]:
train_file = '/work/data/splits/splits_0.05/train.json'
val_file = '/work/data/splits/splits_0.05/val.json'
test_file = '/work/data/splits/splits_0.05/test.json'
import os

path_to_save = '/work/data/models2_6'
os.system(f'mkdir -p {path_to_save}')
path_to_res = '/work/data/results/2_6_2023_tp_graph_5re.txt'
neurons = 64

import

In [9]:
%load_ext autoreload
%autoreload 2
import json
import dgl
from sklearn.metrics import f1_score
import torch as th
import dgl.nn.pytorch as dglnn
import torch.nn as nn
from dgl.dataloading import GraphDataLoader
import torch.nn.functional as F
import networkx as nx
import warnings
from feature_extraction.base_featurizer import BaseFeaturizer
import torch
from graph_construction.tps_graph import create_dummy_dgl_graph, tps_graph
import numpy as np

from dgl_classifier.trainer import GraphDataset,snap_pred, tps_to_dgl, get_clasification_vec,\
tps_graph_const, extract_data, runner


import warnings

from preprocessing.utils import load_BGPS_from_json
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#seed
dgl.seed(1223)

Model

In [11]:
class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.RelGraphConv(in_dim, hidden_dim, 8)
        #self.conv1 = dglnn.GraphConv(in_dim, hidden_ˇdim)
        self.conv2 = dglnn.RelGraphConv(hidden_dim, hidden_dim, 8)
        #self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h, rel_types):
        # Apply graph convolution and activation.
        h = F.relu(self.conv1(g, h, rel_types))
        h = F.relu(self.conv2(g, h, rel_types))
        with g.local_scope():
            g.ndata['node_features'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'node_features')
            return F.softmax( self.classify(hg), dim=1)
model = Classifier(113, 20, 2)

hyperparameters

In [12]:
early_stop =100
model = Classifier(113, neurons, 2)
lr = 0.01
wd = 5e-4
epochs = 100

In [13]:
train_dataloader, val_dataloader, test_dataloader = extract_data(train_file, val_file, test_file, community_no=30, batch_size = 50, verbose=True)

# of catesian products: 0
# of catesian products: 0
# of catesian products: 0
# of predicate feats not in validation 113
# of predicate feats not in test 7
Training dataset size 286
Validation dataset size 96
Test dataset size 96


In [14]:
best_model_path = runner(train_dataloader,val_dataloader,test_dataloader,model, early_stop, lr, wd, epochs, path_to_res, path_to_save=path_to_save)
;

Epoch 1
--------------------------------------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x113 and 64x2)

In [ ]:
import json

In [ ]:
results = json.load(open(path_to_res))

In [ ]:
#print('best f1', np.max(results['val_f1']), np.argmax(results['val_f1']))
print('best f1 val', results['val_f1'][np.argmax(results['val_f1'])])
print('corr f1 train', results['train_f1'][np.argmax(results['val_f1'])])
print('corr f1 test', results['test_f1'][np.argmax(results['val_f1'])])
#print('best f1 train', np.max(results['train_f1']), np.argmax(results['train_f1']))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title('F1 score plot 5% RE threshold')
plt.ylabel('F1')
plt.xlabel('Epochs')
plt.plot(results['train_f1'],'-g', label='Train F1')
plt.plot(results['val_f1'],'-b', label='Val F1')
plt.legend(loc='upper left')
plt.ylim([0.0,1.0])

In [ ]:
list(results.keys())

In [ ]:
plt.title('Precision score plot 5% RE threshold')
plt.ylabel('Precision')
plt.xlabel('Epochs')
plt.plot(results['train_prec'],'-g', label='Train Precision')
plt.plot(results['val_prec'],'-b', label='Val Precision')
plt.legend(loc='upper left')
plt.ylim([0.0,1.0])

In [ ]:
plt.title('Recall score plot 5% RE threshold')
plt.ylabel('Recall')
plt.xlabel('Epochs')
plt.plot(results['train_recall'],'-g', label='Train Recall')
plt.plot(results['val_recall'],'-b', label='Val Recall')
plt.legend(loc='upper left')
plt.ylim([0.0,1.0])

In [ ]:
import torch
model = torch.load(best_model_path)
model.eval()
model

In [ ]:
from dgl_classifier. trainer import predict2samples
print(train_file)
train_data, val_data,test_data = predict2samples(model,train_file, val_file, test_file, community_no=10, verbose=True, clust_verbose=False, clust_load_path='/work/data/confs/May2/pred_clust.json')

In [ ]:
for k in train_data.keys():
    print(train_data[k])
    break

In [ ]:
from dgl_classifier.post import *
train_rt, avg_train = avg_runtime(train_data)
_,_, avg_train_jena,avg_train_blf = avg_runtime_type(train_data)
val_rt, avg_val = avg_runtime(val_data)
_,_, avg_val_jena,avg_val_blf = avg_runtime_type(val_data)
test_rt, avg_test = avg_runtime(test_data)
_,_, avg_test_jena,avg_test_blf = avg_runtime_type(test_data)

In [ ]:
print(avg_train, avg_train_jena, avg_train_blf)
print(avg_val, avg_val_jena, avg_val_blf)
print(avg_test, avg_test_jena, avg_test_blf)